### LIMPIEZA V

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sidetable

plt.rcParams["figure.figsize"] = (10,8)
pd.options.display.max_columns = None

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [8]:
df = pd.read_csv('../datos/bank-additional-full.csv', index_col = 0)

In [9]:
df.sample(3)

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,month_day_week
7935,49,blue-collar,MARRIED,basic.9y,NaN,1.0,0.0,telephone,210,2,999,0,NONEXISTENT,1.4,"94,465","-41,8","4,865","5228,1",no,"['jun', 'mon']"
11208,42,services,MARRIED,high.school,NaN,0.0,0.0,telephone,84,1,999,0,NONEXISTENT,1.4,"94,465","-41,8","4,961","5228,1",no,"['jun', 'thu']"
35503,44,admin.,MARRIED,basic.6y,0.0,0.0,0.0,cellular,217,1,999,1,FAILURE,-1.8,"92,893","-46,2","1,244","5099,1",no,"['may', 'mon']"


In [10]:
df.stb.missing()

,missing,total,percent
default,9016,43170,20.884874
education,1815,43170,4.204309
housing,1031,43170,2.388233
loan,1031,43170,2.388233
job,347,43170,0.803799
marital,85,43170,0.196896
age,0,43170,0.000000
emp.var.rate,0,43170,0.000000
y,0,43170,0.000000
nr.employed,0,43170,0.000000


1- Es el momento de eliminar los nulos:

- Reemplazad los valores nulos del resto de las columnas categóricas por la moda, usando el método SimpleImputer.

    💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [11]:
df

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,month_day_week
0,56,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
1,57,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
2,37,services,MARRIED,high.school,0.0,1.0,0.0,telephone,226,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
3,40,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,151,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
4,56,services,MARRIED,high.school,0.0,0.0,1.0,telephone,307,1,999,0,NONEXISTENT,1.1,"93,994","-36,4","4,857","5191,0",no,"['may', 'mon']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15280,36,admin.,MARRIED,university.degree,NaN,0.0,0.0,cellular,674,3,999,0,NONEXISTENT,1.4,"93,918","-42,7","4,958","5228,1",no,"['jul', 'thu']"
27570,37,unemployed,SINGLE,university.degree,0.0,0.0,1.0,cellular,104,2,999,0,NONEXISTENT,-0.1,"93,2","-42,0","4,021","5195,8",no,"['nov', 'fri']"
17746,47,services,DIVORCED,professional.course,0.0,0.0,0.0,cellular,318,1,999,0,NONEXISTENT,1.4,"93,918","-42,7","4,961","5228,1",no,"['jul', 'tue']"
30574,42,admin.,MARRIED,university.degree,0.0,1.0,0.0,cellular,20,5,999,0,NONEXISTENT,-1.8,"92,893","-46,2","1,354","5099,1",no,"['may', 'mon']"


- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [12]:
def eliminar_nulos(dataframe_completo):
    categoricas = dataframe_completo.select_dtypes(include = 'object')
    numericas = dataframe_completo.select_dtypes(include = np.number)

    # para las categoricas:
    imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
    imputer = imputer.fit(categoricas)
    categoricas_trans = pd.DataFrame(imputer.transform(categoricas), columns = categoricas.columns)
    
    dataframe_completo.drop(categoricas.columns, axis = 1, inplace= True)
    
    dataframe_completo = dataframe_completo.reset_index()
    categoricas_trans = categoricas_trans.reset_index()
    
    dataframe_completo = pd.concat([dataframe_completo, categoricas_trans], axis = 1)
    
    dataframe_completo.drop(['index'], inplace = True, axis = 1)

    # para las numéricas:
    imputer2 = IterativeImputer(n_nearest_features=None, initial_strategy= 'median',imputation_order='ascending')
    imputer2 = imputer2.fit(numericas)
    numericas_trans = pd.DataFrame(imputer2.transform(numericas), columns = numericas.columns)
    
    dataframe_completo.drop(numericas.columns, axis = 1, inplace= True)
    dataframe_completo = dataframe_completo.reset_index()
    numericas_trans = numericas_trans.reset_index()
    
    dataframe_completo = pd.concat([dataframe_completo, numericas_trans], axis = 1)
    
    dataframe_completo.drop(['index'], inplace = True, axis = 1)
    
    return dataframe_completo

In [13]:
df = eliminar_nulos(df)

In [14]:
df.stb.missing()

,missing,total,percent
job,0,43170,0.0
marital,0,43170,0.0
previous,0,43170,0.0
pdays,0,43170,0.0
campaign,0,43170,0.0
duration,0,43170,0.0
loan,0,43170,0.0
housing,0,43170,0.0
default,0,43170,0.0
age,0,43170,0.0


Hemos decidido intentar crear una función que limpie todos los nulos de una sola vez. Para las variables categóricas utilizmamos la categoría más frecuente con el metodo SimpleImputer, mientras que para las numéricas hemos decidido emplear la mediana con IterativeImputer.

- En nuestro data-frame, la columna 'age' no contaba con valores nulos.

- ¿Podríais explicar qué diferencia hay entre estos últimos tres ejercicios?

La diferencia se encuentra en el tipo de variable sobre la que se va a realizar la imputación de nulos, teniendo  que emplear métodos diferentes para los distintos tipos de categorías.

2- Guardad el csv.

In [15]:
df.to_csv('../datos/df_clean_skl.csv')